In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

В качестве тренировки делаю сетку, которая кодирует входную последовательность $x$ и последовательности $y_1...y_k$, и выбирает такой из игреков, представление которого было бы максимально похоже на представление $x$. 

In [323]:
class ToyChooser(nn.Module):
    def __init__(self, hidden_size=64, vocab_size=14, embedding_dim=32, proj_size=128):
        super().__init__()
        RNN = nn.LSTM
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.sent_rnn = RNN(embedding_dim, hidden_size, bidirectional=True)
        self.sent_proj = nn.Linear(hidden_size * 2, proj_size)
        
        self.option_rnn = RNN(embedding_dim, hidden_size, bidirectional=True)
        self.option_proj = nn.Linear(hidden_size * 2, proj_size)
        
        self.mix_mlp = nn.Sequential(
            nn.Linear(3 * proj_size, proj_size), 
            nn.ReLU(), 
            nn.Linear(proj_size, 1),             
        )
    
    def forward(self, sentence, options):
        batch_size = 1
        sent_len = sentence.shape[0]
        x = self.embeddings(sentence).view(sent_len, batch_size,  -1)
        sent_rnn_out, _ = self.sent_rnn(x)
        encoded_sentence = self.sent_proj(sent_rnn_out[-1])[0]
        
        dots = []
        for option in options:
            z = self.embeddings(option).view(option.shape[0], batch_size,  -1)
            opt_rnn_out, _ = self.option_rnn(z)
            encoded_option = self.option_proj(opt_rnn_out[-1])[0]
            #dots.append(torch.cosine_similarity(encoded_sentence, encoded_option, dim=0))
            dots.append(self.mix_mlp(
                torch.cat([encoded_sentence, encoded_option, torch.mul(encoded_sentence, encoded_option)])
            ))
        
        return torch.stack(dots).view(1, -1)

In [324]:
model = ToyChooser()
xx = torch.tensor([1, 2, 3])
model.embeddings(xx).view(1, 3,  -1).shape

torch.Size([1, 3, 32])

In [325]:
model(torch.tensor([1, 2, 3]), torch.tensor([[1,2],[3,4],[5,6]]))

tensor([[0.0284, 0.0528, 0.0423]], grad_fn=<ViewBackward>)

Для начала научу мою модельку выбирать из четырехзначных чисел такое, которое давало бы 10000 в сумме с моим числом. 

Учится очень быстро (хотя числа написаны цифрами)!

In [326]:
import math
import random

def to_digits(number, max_len=2, pad=11, first=12, last=13):
    digits = [int(x) for x in str(number)]
    while len(digits) < max_len:
        digits.append(pad)
    return torch.tensor([first] + digits + [last])

def make_example(min_options=2, max_options=5, total=99):
    x = random.randint(0, total)
    y = total - x
    n_options = random.randint(min_options, max_options)
    options = [y] + [random.randint(0, total) for i in range(n_options - 1)]
    options = [to_digits(z) for z in options]
    return to_digits(x), options, torch.tensor([0]) #torch.tensor([1] + [0] * (n_options - 1))

In [327]:
e = make_example()
e

(tensor([12,  4,  4, 13]),
 [tensor([12,  5,  5, 13]),
  tensor([12,  1,  7, 13]),
  tensor([12,  2,  9, 13])],
 tensor([0]))

In [328]:
scores = model(e[0], e[1])
scores

tensor([[0.0414, 0.0387, 0.0397]], grad_fn=<ViewBackward>)

In [329]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_function = nn.CrossEntropyLoss()

In [330]:
loss_function(scores, torch.tensor([0]))

tensor(1.0971, grad_fn=<NllLossBackward>)

In [331]:
from tqdm.auto import tqdm, trange

Моделька учится быстро и для двухзначных, и для четырехзначных чисел, хотя для четырехзначных точность довольно долго не уходит в 100%. Видимо, сначала моделька долго хитрит и использует только самые правые цифры, что иногда её подводит.

Если увеличить число негативов, перформанс должен вырасти, правда, сравнивать точность надо на батчах с одним и тем же числом негативов, это немножко влом. 

Если сравнивать не косинусную близость, а просто скоры какой-то линейной сетки, то лосс падает быстрее, но, кажется, точность увеличивается не столь драматично. Если в эту линейную сетку добавить покоординатные произведения эмбеддингов, сходимость здорово ускоряется; сеточка оптимизируется очень уверенно. 

In [332]:
PRINT_EVERY = 100

it = 0
tot = 0
act = 0

while True:
    x, z, y = make_example(total=10000, max_options=10)

    optimizer.zero_grad()
    scores = model(x, z)
    l = loss_function(scores, y)
    l.backward()
    optimizer.step()

    it += 1
    tot += l.item()
    act += (scores.detach().numpy().argmax() == 0)
    if it == PRINT_EVERY:
        print(tot/it, act/it)
        it = 0
        tot = 0
        act = 0

1.7480559253692627 0.27
1.6768263405561448 0.23
1.7160321789979935 0.17
1.626733974814415 0.27
1.6945529627799987 0.3
1.7257419884204865 0.31
1.6590426933765412 0.31
1.6574965995550155 0.36
1.5925902646780015 0.44
1.31816881865263 0.47
1.215805449783802 0.44
1.1921605312824248 0.52
0.9778940753638744 0.7
0.9012304529547691 0.59
0.9613528645038605 0.57
0.9437034785747528 0.63
0.7360621747374535 0.7
0.7954510005563498 0.7
0.8864778475463391 0.55
0.7615344820916653 0.63
0.8549534755945206 0.67
0.7611990250647068 0.69
0.7477109287679196 0.66
0.7019262601062656 0.69
0.6603471165150404 0.72
0.7465430384129286 0.65
0.7170129759609699 0.72
0.640943868663162 0.72


KeyboardInterrupt: 

In [321]:
print(model)

ToyChooser(
  (embeddings): Embedding(14, 32)
  (sent_rnn): LSTM(32, 64, bidirectional=True)
  (sent_proj): Linear(in_features=128, out_features=128, bias=True)
  (option_rnn): LSTM(32, 64, bidirectional=True)
  (option_proj): Linear(in_features=128, out_features=128, bias=True)
  (mix_mlp): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1, bias=True)
  )
)
